# はじめに
- 「開始」ボタンを押す前に「パッケージ」から`snowflake-ml-python`をインストールしてください。

# シナリオ
### 0. AISQL を理解する

### 1. LLM の機能を試す
- AISQL を理解する
- 保険金請求書のデータを構造化データへ変換

### 2. AI Agent を使った分析
- ドライバー保険の約款データを使ったRAGの構築
- Text-to-SQLで自動車保険ビジネスにおける様々な部門の課題解決に直結するインサイトの提供

![img](https://lh3.googleusercontent.com/pw/AP1GczMikDMAcJEUxHgauOMoz9HeWhkEZ7fNQJMwirKG6QssMOipCC8Z5GRLdnLAyuMlsfXug71vAcIZFHDaYUMaNHtyaO35aKZTuC9_WPsgIBk6msbvk-U3hs5sKPSGCw7i3oBUeHC4rFDoNRvKCdxXqy4M=w1602-h900-s-no-gm?authuser=0)

## 0. AISQL を理解する

![img](https://lh3.googleusercontent.com/pw/AP1GczPEstMLsFYzhztvUUHigYdBALM3BDLmvKUQKWDMmCQKuebzruOhKZpKoZeMH1kcoVBil8SUwscciE4_z4AtPx6UrRPwfLt6nhJUS1Lg-q8arZy_TAW6-ZYLdGplQedh7XQxLIgcfYobF6Lk8jLAcn10=w1149-h646-s-no-gm?authuser=0)

SQL で AI_COMPLETE 関数を試す

In [ ]:
-- SQL の場合
SELECT AI_COMPLETE('claude-4-sonnet', 'Snowflakeの特徴を50字以内で教えてください。') AS RESPONSE;

Python で AI_COMPLETE 関数を試す

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import ai_complete 

session = get_active_session()
df = session.range(1).select(
    ai_complete('snowflake-arctic', 'Snowflakeの特徴を50字以内で教えてください。').alias('response')
)
df


AI_CLASSIFY 関数によるマルチラベル分類

In [ ]:
SELECT 
    AI_CLASSIFY(
        '高速道路で前の車に追突してしまいました。相手にケガはありませんが車両に損傷があります。',  -- 分類するテキスト
        ['事故報告', '保険料問い合わせ', '契約手続き']  -- 分類カテゴリのリスト
    ) as classification;

AI_SIMILARITY 関数による類似度計算

In [ ]:
SELECT 
    '月々の保険料はいくらですか' as text1,
    '保険料の支払い方法を教えてください' as text2,
    AI_SIMILARITY('月々の保険料はいくらですか', '保険料の支払い方法を教えてください') as similarity_score;

AI_FILTER 関数で事故対応後のフィードバックのポジネガフィルタリング

In [ ]:

-- サンプルテーブル定義（自動車保険の顧客フィードバック）
CREATE OR REPLACE TEMPORARY TABLE INSURANCE_FEEDBACK (
    feedback_id INT,
    customer_id INT,
    feedback_text STRING,
    feedback_date DATE
);

-- サンプルデータ挿入
INSERT INTO INSURANCE_FEEDBACK VALUES
(1, 1001, '事故対応が迅速で、担当者の対応も丁寧でした。安心して任せることができました。', '2024-01-15'),
(2, 1002, '保険料が他社と比べて高いと感じます。割引制度をもっと充実させてほしいです。', '2024-01-20'),
(3, 1003, 'ロードサービスを利用しましたが、到着が遅く困りました。改善をお願いします。', '2024-01-25'),
(4, 1004, '契約手続きがスムーズで、オンラインで簡単に完了できて良かったです。', '2024-02-01'),
(5, 1005, '事故の示談交渉で納得のいかない結果でした。もう少し親身になってほしかった。', '2024-02-05'),
(6, 1006, '24時間サポートがあるので安心です。深夜の事故でもすぐに対応してもらえました。', '2024-02-10'),
(7, 1007, '保険金の支払いが遅いです。手続きをもっと簡素化してください。', '2024-02-15'),
(8, 1008, 'アプリが使いやすく、事故報告も簡単にできました。デジタル化が進んでいて良いです。', '2024-02-20'),
(9, 1009, 'カスタマーサポートの電話がつながりにくいです。回線を増やしてほしい。', '2024-02-25'),
(10, 1010, '等級制度について分かりやすく説明してもらえました。担当者の知識が豊富で助かりました。', '2024-03-01');

-- AI_FILTER関数による条件フィルタリング（事故対応について言及しているフィードバック）
SELECT 
    feedback_id,
    feedback_text,
    AI_FILTER(CONCAT('この顧客フィードバックにポジティブな評価が含まれているか？: ', feedback_text)) as is_positive_feedback
FROM INSURANCE_FEEDBACK
WHERE AI_FILTER(CONCAT('事故対応について言及されているか？: ', feedback_text)) = TRUE
LIMIT 10;

AI_AGG 関数で全体のサービス満足度要約

In [ ]:
SELECT 
    AI_AGG(
        feedback_text, 
        'サービス全般に関する満足度について簡単に要約してください'
    ) as overall_service_insights
FROM INSURANCE_FEEDBACK;

## 1. LLM の機能を試す

![img](https://lh3.googleusercontent.com/pw/AP1GczOsDabpXc1oDa23mXAvH6K_hkITlrTuNVy9qTbOY4sXZJDjEHQgbIm6Z7LXnvpflVCjCmjhB7mIqHkydFWJDOGuvHkEUywuTcp3ULLlqbiELYtUe-VH0NA7O7y-ibmNeo-izQhIYyLKbC65gNXDZJmS=w1602-h896-s-no-gm?authuser=0)

## 1.2 保険金請求書のデータを構造化データへ変換

[サンプルデータ](https://www.sompo-japan.co.jp/-/media/SJNK/files/covenanter/acontact/support/transportins/contents3/FORM_1.pdf?la=ja-JP)を利用

![img](https://lh3.googleusercontent.com/pw/AP1GczNM1fOPqa0dZ1ulQjNzNxz-DJWDebuYy3tlcimN_f_fT4qWTXkAosM8NotOnyJM4fzL4Mlj5o00SbEr-Bmr4LZ2WBVf2DWzXB7S4d7M6rEOYeS-Xp5P1K1Q4xEI-bmPK9z6UuRx2FiBa3EgEX0xTJgJ=w896-h1229-s-no-gm?authuser=0)

In [ ]:
-- テキスト抽出
CREATE OR REPLACE TABLE handson.car_insurance.extract_table AS
SELECT
  relative_path AS file_name,
  AI_EXTRACT(
    file => TO_FILE('@handson.car_insurance.car_insurance_stage', relative_path),
    responseFormat => [
      ['accident_response', '損害貨物への対応は何ですか？'],
      ['name',      '氏名は何ですか？'],
      ['registration_number', '登録番号(ナンバープレート)は何ですか？'],
      ['security_number', '証券番号は何ですか？'],
      ['accident_cause',       '事故の原因は何ですか？'],
      ['billing_date',  '請求日はいつですか？']
    ]
  ) AS document_extracts
FROM DIRECTORY(@handson.car_insurance.car_insurance_stage);


In [ ]:
SELECT * FROM handson.car_insurance.extract_table;

In [ ]:
CREATE OR REPLACE TABLE car_claim_form AS
WITH long AS (
  SELECT
      t.file_name,
      f.value::string AS field_name,
      NULLIF(GET(t.document_extracts:response, f.value)::string, 'None') AS field_value
  FROM handson.car_insurance.extract_table t,
       LATERAL FLATTEN(input => OBJECT_KEYS(t.document_extracts:response)) f
)
SELECT *
FROM long
PIVOT (
  MAX(field_value) FOR field_name
  IN ('accident_response', 'name', 'registration_number', 'security_number', 'accident_cause', 'billing_date')
) AS p (file_name, accident_response, name, registration_number, security_number, accident_cause, billing_date);

In [ ]:
SELECT * FROM car_claim_form;

## 2.1 ドライバー保険の約款データを使ったRAGの構築

![img](https://lh3.googleusercontent.com/pw/AP1GczOySI9gnWJCcNzeBcO-Yf-c1iril-Hd4xMllM_WcEZT0nzxH7PyDeuQmj7Q-CYNeWyS0h0WgYMWTsr4Q5cVjjuzjzRgtKNwa4pQ_vWC1dxEzX8_tagVAH-SnrJDl0p89HaBpLwK5PzGj0W9zpiVxglE=w1602-h900-s-no-gm?authuser=0)

### ドライバー保険の約款を保管するステージの作成

In [ ]:
CREATE OR REPLACE STAGE handson.car_insurance.terms_and_conditions
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

### ドライバー保険の約款をアップロード（GUIで実施）

1. データ -> データを追加を選択
2. タイルを選択: テーブルにデータをロード

### PARSE_DOCUMENT を使った PDF データの抽出
- PARSE_DOCUMENT OCR （デフォルト） モードは、テキスト量の多いドキュメントからのテキスト抽出に最適化されています。これは、強力なセマンティック構造を持たないドキュメントから、迅速、簡単、かつ効果的にテキストを抽出するために推奨されるオプションです。

- PARSE_DOCUMENT LAYOUT （オプション） モードは、テーブルのようなテキストとレイアウト要素の抽出に最適化されています。これは、検索情報システムや大規模言語モデル（LLM）推論を最適化し、ドキュメントナレッジベースのコンテキストを改善するために推奨されるオプションです。たとえば、 LAYOUT 要素を使用してテキストセクションを分離し、より的を絞ったエンティティ抽出作業を行うことができます。

[ドキュメント](https://docs.snowflake.com/ja/user-guide/snowflake-cortex/parse-document) (最新の情報は英語ドキュメントを参照)

![img](https://lh3.googleusercontent.com/pw/AP1GczNO_lTZjDwPtnGpwFZrImzwRYhs3W1TbU1otTqW55cdEb3v8qZvXBvseiFDUS00AdQ0Lilzc25TQ5kM9mma9jHbYtjUILW0wJhHJ9VxcJQuTyBK0jr09P0jelBU6C0b-pQNbdjaQ9smOWreI7itusRB=w1602-h896-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE TABLE handson.car_insurance.parsed_terms_and_conditions AS SELECT

relative_path, 
BUILD_STAGE_FILE_URL('@handson.car_insurance.terms_and_conditions', relative_path) as file_url,
 TO_File(BUILD_STAGE_FILE_URL('@handson.car_insurance.terms_and_conditions', relative_path) ) file_object,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
                                @handson.car_insurance.terms_and_conditions,
                                relative_path,
                                {'mode':'LAYOUT'}
                                ):content::string as Content
from directory(@handson.car_insurance.terms_and_conditions) 
where relative_path ilike '%.pdf';

In [ ]:
SELECT * FROM handson.car_insurance.parsed_terms_and_conditions;

In [ ]:
SELECT
    relative_path,
    file_url,
    REGEXP_SUBSTR(relative_path, '[^/]+$') as title, -- Extract filename as title
    content
FROM handson.car_insurance.parsed_terms_and_conditions

SPLIT_TEXT_RECURSIVE_CHARACTER を使ったチャンクの作成

本ハンズオンでは以下を設定
- 分割対象: contentカラムのテキストを分割
- フォーマット: 'markdown' - ヘッダー、コードブロック、テーブルで分割
- チャンクサイズ: 2500文字 - 各チャンクの最大文字数
- 重複サイズ: 500文字 - 隣接するチャンク間で重複させる文字数
- 処理結果: 2500文字以下のチャンクに分割され、500文字ずつ重複

[ドキュメント](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex) (最新の情報は英語ドキュメントを参照)

![img](https://lh3.googleusercontent.com/pw/AP1GczOukcbNVYHh_JLAbGE2C6aYZZZWDbeJ7bxF7KXWxMMV3_f6AVOTzfuqVtH_rdiEqQgBy8aiXdBxgPmCPMUYBWYUmsdf_B9kp3Cppcj9ooh2rhX_0AC2NEI2ROWGpXVnTEIrkAe5v0eOjb7kGgwJVfli=w1602-h885-s-no-gm?authuser=0)

In [ ]:
-- チャンク分割用の新しいテーブルの作成
CREATE OR REPLACE TABLE handson.car_insurance.parsed_terms_and_conditions_chunks (
    relative_path VARCHAR,
    file_url VARCHAR,
    title VARCHAR,
    CHUNK VARCHAR
);

-- 元のテーブルからデータを取得してチャンク分割
INSERT INTO handson.car_insurance.parsed_terms_and_conditions_chunks (relative_path, file_url, title, CHUNK)
SELECT
    relative_path,
    file_url,
    REGEXP_SUBSTR(relative_path, '[^/]+$') as title, 
    c.value AS CHUNK
FROM
    handson.car_insurance.parsed_terms_and_conditions_2,  -- 実際に作成されたテーブル名
    LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
        content,
        'markdown',
        2500,
        500
    )) c;

チャンクデータの確認

In [ ]:
SELECT * FROM handson.car_insurance.parsed_terms_and_conditions_chunks;

### Cortex Search を利用しコンテキストを理解可能なサービスを作成

![img](https://lh3.googleusercontent.com/pw/AP1GczN33sZa58DW5WZnqcjAiLBgFXg2SH1qKlMXXCfNLGADZjix7BYwIYAQ8GI_vssc4Q8XnyevRP0lbhh-nwsDTzUeR05BjvJh9mjRvIZmBnpv0QllSabu7W49EHtS31LBO3UHlKtdSThF3XG6Na4re_DL=w1602-h900-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE handson.car_insurance.terms_and_conditions_service
    ON chunk
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = handson_wh
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            title,
            chunk
        FROM handson.car_insurance.parsed_terms_and_conditions_chunks
    );

アカウント内で作成された Cortex Search サービスを確認

In [ ]:
SHOW CORTEX SEARCH SERVICES IN ACCOUNT;

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'handson.car_insurance.terms_and_conditions_service',
      '{
         "query": "事故が発生した場合の対応手順を教えてください。",
         "columns": ["chunk", "title", "file_url"],
         "limit": 5
      }'
  )
)['results'] as results;

In [ ]:
SELECT 
    '事故対応手順検索' as search_type,
    (results.index + 1) as result_rank,
    results.value['title']::string as document_title,
    results.value['chunk']::string as content,
    LENGTH(results.value['chunk']::string) as content_length,
    -- results.value['file_url']::string as source_file_url
FROM (
    SELECT 
        PARSE_JSON(
            SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
                'handson.car_insurance.terms_and_conditions_service',
                '{
                    "query": "事故が発生した場合の対応手順を教えてください。",
                    "columns": ["chunk", "title", "file_url"],
                    "limit": 5
                }'
            )
        )['results'] as results_array
),
LATERAL FLATTEN(input => results_array) as results
ORDER BY results.index;

## Text-to-SQLで自動車保険ビジネスにおける様々な部門の課題解決に直結するインサイトの提供
- SQLでセマンティックビューを作成し、GUIで動作を確認する

![img](https://lh3.googleusercontent.com/pw/AP1GczOO9VK2oFroeFm8PYKnGhGAL7W2xXdiz_jjgYEgsX_iczpUI_DxnzUeTWuEk0vjNWS5BAbHBsl5TaGC-c__lyNA0uAvHEgD01eg1nq9_N0rB0ZxGl2T4x5KdxR9E4KS3RSCtS2qqlakVh4M_6-8KZZB=w1602-h895-s-no-gm?authuser=0)

![img](https://lh3.googleusercontent.com/pw/AP1GczPaGaGvSm2iGb09AHpaDREOX0-4Ep37I944iC6bOhrYb7pTQFexIPKLVOPNLVSOLQ8oSkF3G36IUSSH741lcMoyGwXwuXvSbnY0GSnIoZFbupj1kSWs03k99BawsxcWtCH3y2VtvAUIcfRC9epwCnhW=w1634-h920-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW INSURANCE_CLAIMS_SEMANTIC_VIEW
    tables (
        CLAIMS as CLAIMS primary key (CLAIM_ID) 
            with synonyms=('保険金請求','請求','事故') 
            comment='保険金請求に関する中心的なファクトテーブル',
        POLICIES as POLICIES primary key (POLICY_ID) 
            with synonyms=('保険契約','契約','ポリシー') 
            comment='保険契約者と車両に関するディメンションテーブル',
        REPAIR_SHOPS as REPAIR_SHOPS primary key (SHOP_ID) 
            with synonyms=('修理工場','工場','パートナー工場') 
            comment='提携している修理工場に関するディメンションテーブル',
        REPAIR_ORDERS as REPAIR_ORDERS primary key (ORDER_ID) 
            with synonyms=('修理指示','修理明細','作業指示') 
            comment='各請求に対する修理作業の明細データ'
    )
    relationships (
        CLAIMS_TO_POLICIES as CLAIMS(POLICY_ID) references POLICIES(POLICY_ID),
        CLAIMS_TO_SHOPS as CLAIMS(REPAIR_SHOP_ID) references REPAIR_SHOPS(SHOP_ID),
        ORDERS_TO_CLAIMS as REPAIR_ORDERS(CLAIM_ID) references CLAIMS(CLAIM_ID)
    )
    facts (
        CLAIMS.claim_record as 1 
            comment='請求レコード数。件数のカウントに使用',
        CLAIMS.approved_cost_yen as APPROVED_COST_YEN 
            comment='承認された修理費用（円）',
        CLAIMS.estimate_cost_yen as ESTIMATE_COST_YEN 
            comment='見積もり修理費用（円）',
        CLAIMS.paid_to_date_yen as PAID_TO_DATE_YEN 
            comment='支払済み金額（円）',
        POLICIES.policy_record as 1 
            comment='契約レコード数。件数のカウントに使用',
        POLICIES.annual_premium_yen as ANNUAL_PREMIUM_YEN 
            comment='年間保険料（円）',
        REPAIR_ORDERS.repair_cost as LINE_TOTAL_AFTER_ADJ_YEN
            comment='修理明細の合計金額（調整後、円）',
        REPAIR_ORDERS.labor_hours as LABOR_HOURS
            comment='作業時間（時間）'
    )
    dimensions (
        -- CLAIMS テーブルのディメンション
        CLAIMS.incident_date as INCIDENT_DATE 
            with synonyms=('事故日','発生日') 
            comment='事故が発生した日付',
        CLAIMS.incident_year as YEAR(incident_date) 
            with synonyms=('事故年','発生年') 
            comment='事故が発生した年',
        CLAIMS.incident_month as MONTH(incident_date) 
            with synonyms=('事故月','発生月') 
            comment='事故が発生した月',
        CLAIMS.claim_status as CLAIM_STATUS 
            with synonyms=('請求ステータス','ステータス') 
            comment='保険金請求の現在のステータス (Open, Approved, Paid, Denied)',
        CLAIMS.loss_cause as LOSS_CAUSE 
            with synonyms=('事故原因','原因') 
            comment='損害の原因 (Collision, Weather, Theftなど)',
        CLAIMS.severity as SEVERITY 
            with synonyms=('損害の大きさ','深刻度') 
            comment='損害の深刻度 (Minor, Moderate, Severe, Total Loss)',
        CLAIMS.incident_prefecture as INCIDENT_PREFECTURE 
            with synonyms=('事故発生都道府県','事故現場の都道府県') 
            comment='事故が発生した都道府県',
        CLAIMS.fraud_score as FRAUD_SCORE
            with synonyms=('不正スコア', '詐欺スコア')
            comment='請求の不正行為の可能性を示すスコア',
        -- POLICIES テーブルのディメンション
        POLICIES.vehicle_make as VEHICLE_MAKE 
            with synonyms=('自動車メーカー','メーカー') 
            comment='車両のメーカー',
        POLICIES.vehicle_model as VEHICLE_MODEL
            with synonyms=('車種','モデル')
            comment='車両のモデル名',
        POLICIES.vehicle_age_band as VEHICLE_AGE_BAND 
            with synonyms=('車齢帯','車両年式') 
            comment='車両の年式帯 (Newer, Mid, Older)',
        POLICIES.vehicle_use as VEHICLE_USE
            with synonyms=('車両用途','使用目的')
            comment='車両の主な使用目的 (Personal, Commute, Commercial)',
        POLICIES.driver_age as DRIVER_AGE
            with synonyms=('運転者年齢','ドライバー年齢')
            comment='契約時の運転者の年齢',
        POLICIES.policy_status as POLICY_STATUS 
            with synonyms=('契約ステータス') 
            comment='保険契約のステータス (Active, Lapsed, Cancelled)',
        POLICIES.registered_prefecture as REGISTERED_PREF
            with synonyms=('登録都道府県')
            comment='車両が登録されている都道府県',
        -- REPAIR_SHOPS テーブルのディメンション
        REPAIR_SHOPS.shop_name as SHOP_NAME 
            with synonyms=('修理工場名','工場名') 
            comment='修理工場の名称',
        REPAIR_SHOPS.shop_prefecture as PREFECTURE 
            with synonyms=('工場都道府県','工場所在地') 
            comment='修理工場の都道府県',
        REPAIR_SHOPS.partner_tier as PARTNER_TIER 
            with synonyms=('パートナーランク','ティア') 
            comment='修理工場のパートナーランク (A, B, C)',
        REPAIR_SHOPS.shop_rating as RATING
            with synonyms=('工場評価','評価')
            comment='修理工場の評価スコア',
        -- REPAIR_ORDERS テーブルのディメンション
        REPAIR_ORDERS.work_start_date as WORK_START_DATE
            with synonyms=('修理開始日')
            comment='修理作業の開始日',
        REPAIR_ORDERS.work_end_date as WORK_END_DATE
            with synonyms=('修理完了日')
            comment='修理作業の完了日'
    )
    metrics (
        CLAIMS.TOTAL_CLAIMS as COUNT(claims.claim_record) 
            with synonyms=('総請求件数','合計請求件数')
            comment='すべての保険金請求の合計件数',
        CLAIMS.TOTAL_APPROVED_COST as SUM(claims.approved_cost_yen) 
            with synonyms=('総承認費用','合計承認費用')
            comment='承認された修理費用の合計額',
        CLAIMS.AVERAGE_APPROVED_COST as AVG(claims.approved_cost_yen) 
            with synonyms=('平均承認費用')
            comment='承認された修理費用の平均額',
        CLAIMS.TOTAL_PAID_AMOUNT as SUM(claims.paid_to_date_yen)
            with synonyms=('総支払額','合計支払額')
            comment='支払われた保険金の合計額',
        POLICIES.TOTAL_POLICIES as COUNT(policies.policy_record) 
            with synonyms=('総契約件数','合計契約件数')
            comment='保険契約の総件数',
        POLICIES.TOTAL_PREMIUM_REVENUE as SUM(policies.annual_premium_yen) 
            with synonyms=('総保険料収入','合計保険料')
            comment='年間保険料の合計額',
        REPAIR_ORDERS.TOTAL_REPAIR_COST as SUM(repair_orders.repair_cost)
            with synonyms=('総修理費用','合計修理費用')
            comment='修理にかかった費用の総額'
    )
    COMMENT='自動車保険の契約、請求、修理に関する分析のためのセマンティックビュー';

## 作成したオブジェクト(Cortex Search, Cortex Analyst)を使って Cortex Agent を作成する
- GUI で実施

![img](https://lh3.googleusercontent.com/pw/AP1GczMrL3z5ohe70vavzro_T5d76yQiLwy7zKxUy2lB-UbQXxV_hi19CeRP8EtCV7Ih7ufIPo6MWMhki8U0CclmiWaIfRc13PeNOshMx9Z7aP9Vi0D7u8spBxJas5CiOTM6_DAqL8GpdSdYK9mEnFJ_bapq=w1602-h900-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE AGENT snowflake_intelligence.agents.driver_insurance_chatbot_agent
WITH PROFILE='{ "display_name": "driver_insurance_agent" }'
    COMMENT=$$ 自動車保険に関する質問に回答するエージェントです。 $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "あなたは自動車保険に関するのデータマートにアクセスできるデータアナリストです。ユーザーが日付範囲を指定しない場合は、2025年と仮定してください。すべてのドメインからのデータを活用して、ユーザーの質問を分析し回答してください。可能であれば視覚化を提供してください。トレンドラインは線グラフをデフォルトとし、カテゴリは棒グラフを使用してください。",
    "orchestration": "既知のエンティティに対してcortex searchを使用し、詳細な分析のためにcortex analystに結果を渡してください。",
    "sample_questions": [
      {
        "question": "事故が発生した都道府県ごとに、平均承認費用を比較して、高い順に並べて可視化してください"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "DRIVER INSURANCE SYSTEM DATAMART",
        "description": "お客様の自動車保険に関するデータをクエリすることをユーザーに許可します。"
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "DRIVER INSURANCE SEARCH",
        "description": "このツールはドライバー保険の約款データを検索するために使用します。"
      }
    }
  ],
  "tool_resources": {
    "DRIVER INSURANCE SYSTEM DATAMART": {
      "semantic_view": "handson.car_insurance.insurance_claims_semantic_view"
    },
    "DRIVER INSURANCE SEARCH": {
      "id_column": "file_url",
      "max_results": 5,
      "name": "handson.car_insurance.terms_and_conditions_service",
      "title_column": "title"
    }
  }
}
$$;

## Snowflake Intelligence を使った AI Agent チャットボットの作成
- GUIで作成

![img](https://lh3.googleusercontent.com/pw/AP1GczM9D1krNnRYylHontiWJ8hSQdf7dmyH5F4ej4_3SDZCTKNZjStUnKNm7_OFGyMi3XyCd05xjM5_VSdRfsjfaLt2n3ckxVP6EbtlkKUFFrZXoRM-ZW7TyUTeXaaY_0ZzHiDJT88P1J4yoL3b-sA-3Hke=w1602-h897-s-no-gm?authuser=0)